In [3]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
train_df=pd.read_csv('train.csv')
train_df

,x,y,color,marker
0,-0.765775,1.006452,red,^
1,0.229016,0.094320,red,^
2,0.189987,1.589141,red,^
3,0.580085,0.520488,red,^
4,0.292287,0.317852,red,^
...,...,...,...,...
5995,0.795819,2.726449,purple,*
5996,-0.362444,3.184180,purple,*
5997,0.638764,2.520736,purple,*
5998,0.845822,3.345033,purple,*


We have 6 colors and 3 markers, so we make a one-hot

In [33]:
one_hot_color=pd.get_dummies(train_df.color,drop_first=True).values
one_hot_marker=pd.get_dummies(train_df.marker).values

In [22]:
labels=np.concatenate((one_hot_color, one_hot_marker),axis=1)
labels

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 1, 0, 0]], dtype=uint8)

## Model preparation:

In [34]:
model=keras.Sequential([#allows us to list layers
    keras.layers.Dense(32, input_shape=(2,),activation='relu'),#32 layers, inputshape of 2 because its x and y.
#     keras.layers.Dropout(0.2),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dense(5, activation='sigmoid') #6 for the binary we could use sigmoid or softmax
]) 

model.compile(optimizer='adam',
             loss=keras.losses.BinaryCrossentropy(), #BinaryCrossenropy predicts each position independently (because we could have more than one 1)
#logits means function operates on unscaled output of earlier layers and the relative scale to understand the unit is linear.
#we use logits because our values(x and y) are not between 0 and 1
              metrics=['accuracy']
             ) 


we need to change our X and Y into one variable

In [35]:
x=np.column_stack((train_df.x.values,train_df.y.values)) #This pairs x and y columns together in one value

Note: We need to shuffle because highly correlated points are next to each other.
we also need to use the same seed for both to not mess up our data

In [36]:
from sklearn.preprocessing import MinMaxScaler
np.random.RandomState(seed=42).shuffle(x)
np.random.RandomState(seed=42).shuffle(one_hot_color)

x=MinMaxScaler().fit_transform(x)
# labels=MinMaxScaler().fit_transform(labels)


## Fitting training data:

In [37]:
model.fit(x,one_hot_color,batch_size=4,epochs=10)

Epoch 1/10
1500/1500 [==============================] - 2s 663us/step - loss: 0.3491 - accuracy: 0.4532
Epoch 2/10
1500/1500 [==============================] - 1s 605us/step - loss: 0.0800 - accuracy: 0.8029
Epoch 3/10
1500/1500 [==============================] - 1s 599us/step - loss: 0.0511 - accuracy: 0.8112
Epoch 4/10
1500/1500 [==============================] - 1s 601us/step - loss: 0.0395 - accuracy: 0.8089
Epoch 5/10
1500/1500 [==============================] - 1s 605us/step - loss: 0.0350 - accuracy: 0.8048
Epoch 6/10
1500/1500 [==============================] - 1s 606us/step - loss: 0.0302 - accuracy: 0.8149
Epoch 7/10
1500/1500 [==============================] - 1s 608us/step - loss: 0.0253 - accuracy: 0.8160
Epoch 8/10
1500/1500 [==============================] - 1s 624us/step - loss: 0.0255 - accuracy: 0.8172
Epoch 9/10
1500/1500 [==============================] - 1s 664us/step - loss: 0.0270 - accuracy: 0.8012
Epoch 10/10
1500/1500 [==============================] - 1s 686u

## Test data:

In [ ]:
test_df=pd.read_csv('test.csv')
test_df.color=test_df.color.map(d)
test_x=np.column_stack((test_df['x'].values,test_df.y.values))


In [ ]:
model.evaluate(test_x,test_df.color.values)

In [ ]:
model.predict(np.array([[0,3]])) #Look on graph and find 0,3, see if the color matches
#np.round(model.predict(np.array([[0,3]]))) looks better